In [1]:
!pip install --upgrade git+https://github.com/flairNLP/flair.git

  Cloning https://github.com/flairNLP/flair.git to /tmp/pip-req-build-d5c6wfoe
  Running command git clone -q https://github.com/flairNLP/flair.git /tmp/pip-req-build-d5c6wfoe
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for flair: filename=flair-0.5-cp36-none-any.whl size=148939 sha256=087edd58e35d7473036a5c667cab9327ec2dc86dee5289506c689f8934f84ad2
  Stored in directory: /tmp/pip-ephem-wheel-cache-4wzk3ps9/wheels/84/82/73/d2b3b59b7be74ea05f2c6d64132efe27df52daffb47d1dc7bb
Successfully built flair
  Found existing installation: flair 0.5
    Uninstalling flair-0.5:
      Successfully uninstalled flair-0.5


In [0]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings, DocumentRNNEmbeddings, BertEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
from flair.datasets import CSVClassificationCorpus
from flair.data import Corpus
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#data_folder = "./drive/My Drive/capstone/data/"
data_folder = "/content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotated_sample_for_training/"

### First Stage (Train on benchmark dataset)

In [0]:
benchmark = pd.read_csv(data_folder + "combined_benchmark.csv")

In [6]:
benchmark = benchmark[['label', 'text']]
benchmark.head()

,label,text
0,0,Why not subscribe to the magazine ?
1,-1,"Tornio Works employs 2,300 of whom more than 1..."
2,1,"The move is aimed at boosting sales , cost-eff..."
3,0,"As a result of the merger , the largest profes..."
4,-1,18 March 2010 A leakage in the gypsum pond was...


#### Create train, dev and test set

In [0]:
#benchmark = benchmark.sample(frac=1) # if not set random state, everytime has different training result

benchmark = benchmark.sample(frac=1, random_state=42)


benchmark.iloc[0:int(len(benchmark)*0.8)].to_csv(data_folder + 'train.csv', sep='\t', index = False, header = False)
benchmark.iloc[int(len(benchmark)*0.8):int(len(benchmark)*0.9)].to_csv(data_folder + 'test.csv', sep='\t', index = False, header = False)
benchmark.iloc[int(len(benchmark)*0.9):].to_csv(data_folder + 'dev.csv', sep='\t', index = False, header = False)

In [8]:
benchmark.head()

,label,text
892,1,About Elcoteq Elcoteq SE is a leading electron...
1105,-1,U.S. goods trade deficit deteriorates; factory...
413,0,Product coverage : baked goods ; biscuits ; br...
522,0,Tyrv+Æinen is of the opinion that the airline ...
1036,-1,Wall St. Week Ahead: U.S. stock reign may not ...


In [0]:

# benchmark.iloc[0:int(len(benchmark)*0.8)].to_csv(data_folder + 'train.csv', index = False, header = False)
# benchmark.iloc[int(len(benchmark)*0.8):int(len(benchmark)*0.9)].to_csv(data_folder + 'test.csv', index = False, header = False)
# benchmark.iloc[int(len(benchmark)*0.9):].to_csv(data_folder + 'dev.csv', index = False, header = False)

In [0]:
#pd.read_csv("/content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotated_sample_for_training/dev.csv")

#### Build corpus

In [11]:
#corpus = NLPTaskDataFetcher.load_classification_corpus(Path(data_folder), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')
column_name_map = {1: "text", 0: "label_topic"}

corpus: Corpus = CSVClassificationCorpus(data_folder,
                                         column_name_map,
                                         skip_header=False, #no header in kaggle data
                                         delimiter='\t',    # comma separated rows
                                         train_file='train.csv', ## passing in file names manuelly when it can't auto detect
                                         dev_file='dev.csv',
                                         test_file='test.csv'
)

2020-05-26 23:19:52,182 Reading data from /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotated_sample_for_training
2020-05-26 23:19:52,183 Train: /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotated_sample_for_training/train.csv
2020-05-26 23:19:52,184 Dev: /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotated_sample_for_training/dev.csv
2020-05-26 23:19:52,186 Test: /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotated_sample_for_training/test.csv


#### Create word embeddings

In [12]:
word_embeddings = [BertEmbeddings(), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]

#word_embeddings = [BertEmbeddings()]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated method __init__. (Use 'TransformerWordEmbeddings' for all transformer-based word embeddings) -- Deprecated since version 0.4.5.
  """Entry point for launching an IPython kernel.


#### First Stage Fine-tuning

In [13]:
document_embeddings = DocumentLSTMEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)
trainer = ModelTrainer(classifier, corpus)
trainer.train(data_folder, max_epochs=10)

2020-05-26 23:19:59,869 Computing label dictionary. Progress:


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated method __init__. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  """Entry point for launching an IPython kernel.
100%|██████████| 1314/1314 [00:01<00:00, 1169.01it/s]

2020-05-26 23:20:01,318 [b'1', b'-1', b'0']
2020-05-26 23:20:01,332 ----------------------------------------------------------------------------------------------------
2020-05-26 23:20:01,336 Model: "TextClassifier(
  (document_embeddings): DocumentLSTMEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): BertEmbeddings(
        (model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=Tru

2020-05-26 23:20:04,100 epoch 1 - iter 3/37 - loss 1.44395936 - samples/sec: 68.68
2020-05-26 23:20:18,138 epoch 1 - iter 6/37 - loss 1.39971318 - samples/sec: 75.61
2020-05-26 23:20:30,443 epoch 1 - iter 9/37 - loss 1.26294792 - samples/sec: 75.18
2020-05-26 23:20:44,189 epoch 1 - iter 12/37 - loss 1.32099261 - samples/sec: 72.24
2020-05-26 23:20:56,804 epoch 1 - iter 15/37 - loss 1.32809993 - samples/sec: 74.58
2020-05-26 23:21:08,920 epoch 1 - iter 18/37 - loss 1.26090499 - samples/sec: 77.14
2020-05-26 23:21:21,621 epoch 1 - iter 21/37 - loss 1.25372351 - samples/sec: 76.97
2020-05-26 23:21:34,145 epoch 1 - iter 24/37 - loss 1.22728333 - samples/sec: 72.79
2020-05-26 23:21:46,480 epoch 1 - iter 27/37 - loss 1.18865638 - samples/sec: 78.49
2020-05-26 23:21:58,849 epoch 1 - iter 30/37 - loss 1.16654052 - samples/sec: 84.33
2020-05-26 23:22:11,401 epoch 1 - iter 33/37 - loss 1.15726749 - samples/sec: 81.30
2020-05-26 23:22:23,828 epoch 1 - iter 36/37 - loss 1.16265203 - samples/sec: 8

{'dev_loss_history': [0.9083929061889648,
  0.8781045079231262,
  0.8748133778572083,
  0.9871878623962402,
  0.9453868865966797,
  1.224395751953125,
  0.9033380746841431,
  0.8314997553825378,
  0.859683632850647,
  0.9503604769706726],
 'dev_score_history': [0.7414965986394558,
  0.7414965986394558,
  0.7278911564625851,
  0.7233560090702947,
  0.7369614512471655,
  0.7097505668934241,
  0.7551020408163265,
  0.764172335600907,
  0.7777777777777778,
  0.7687074829931972],
 'test_score': 0.7397260273972602,
 'train_loss_history': [1.1566761973741893,
  0.9108730457924508,
  0.8433868740056012,
  0.769005719068888,
  0.699600669983271,
  0.6829892888262465,
  0.5233222890544582,
  0.5122124190266067,
  0.4774902181045429,
  0.44998007328123657]}

In [0]:
# finetune phase1 result with Bert+Flair embeddings
#MACRO_AVG: acc 0.7260273972602739 - f1-score 0.5739219483960016

## another run -- finetune phase1 result with Bert+Flair -- 4pm
# MACRO_AVG: acc 0.8493150684931506 - f1-score 0.7757725232278944

In [0]:
# phase1 result with Bert embeddings only . May26 2pm

#MACRO_AVG: acc 0.8036529680365296 - f1-score 0.6885983753954866

In [16]:
from flair.data import Sentence
# create example sentence
sentence = Sentence("Canada's growth weakens as investment drops, consumers fade")

# predict class and print
classifier.predict(sentence)

print(sentence.labels)

[-1 (0.988)]


In [17]:
#  access the Sentence objects in each split directly
print(corpus.test[0])

Sentence: "Finnish metal products company Componenta Oyj net profit went slightly down to 25.1 mln euro ( $ 40.2 mln ) for the first half of 2008 from 25.4 mln euro ( $ 40.7 mln ) for the same period of 2007 ."   [− Tokens: 42  − Sentence-Labels: {'class': [-1 (1.0)]}]


In [18]:
print(corpus.train[0])

Sentence: "About Elcoteq Elcoteq SE is a leading electronics manufacturing services ( EMS ) company in the communications technology field ."   [− Tokens: 20  − Sentence-Labels: {'class': [1 (1.0)]}]


In [19]:
#outputs detailed information on the dataset, each split, and the distribution of class labels.
stats = corpus.obtain_statistics()
print(stats)

{
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 1168,
        "number_of_documents_per_class": {
            "1": 376,
            "-1": 400,
            "0": 392
        },
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 36583,
            "min": 4,
            "max": 69,
            "avg": 31.321061643835616
        }
    },
    "TEST": {
        "dataset": "TEST",
        "total_number_of_documents": 146,
        "number_of_documents_per_class": {
            "-1": 53,
            "1": 55,
            "0": 38
        },
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 4807,
            "min": 6,
            "max": 61,
            "avg": 32.92465753424658
        }
    },
    "DEV": {
        "dataset": "DEV",
        "total_number_of_documents": 147,
        "number_of_documents_per_class": {
            "-1": 41,
            "0": 52,
            "1": 54
        },


### Second Stage (train on hand annotated datasets)

#### Build corpus

In [20]:
#new_data_folder = './drive/My Drive/capstone/data/second_stage/'
new_data_folder = '/content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotations_bnn_cbc/oversampled/'
new_column_name_map = {5: "text", 4: "label_topic"}

corpus: Corpus = CSVClassificationCorpus(new_data_folder,
                                         new_column_name_map,
                                         skip_header=True, #no header in kaggle data
                                         delimiter=',',    # comma separated rows
                                         train_file='GDP_train_df_oversampled.csv',
                                         dev_file = 'GDP_eva_df.csv',
                                         test_file = 'GDP_eva_df.csv'
)

2020-05-26 23:46:15,380 Reading data from /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotations_bnn_cbc/oversampled
2020-05-26 23:46:15,381 Train: /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotations_bnn_cbc/oversampled/GDP_train_df_oversampled.csv
2020-05-26 23:46:15,381 Dev: /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotations_bnn_cbc/oversampled/GDP_eva_df.csv
2020-05-26 23:46:15,382 Test: /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotations_bnn_cbc/oversampled/GDP_eva_df.csv


#### Second Stage fine-tuning

In [21]:
# benchmark_embeddings = FlairEmbeddings(data_folder + 'best-model.pt')
benchmark_classifier = TextClassifier.load(data_folder + 'best-model.pt')

2020-05-26 23:46:15,397 loading file /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotated_sample_for_training/best-model.pt


In [22]:
trainer = ModelTrainer(benchmark_classifier, corpus)
trainer.train(new_data_folder, max_epochs=10)

2020-05-26 23:46:16,723 ----------------------------------------------------------------------------------------------------
2020-05-26 23:46:16,729 Model: "TextClassifier(
  (document_embeddings): DocumentLSTMEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): BertEmbeddings(
        (model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=True)
                    (key): Linear(in_feat

{'dev_loss_history': [3.112643003463745,
  2.512822151184082,
  1.371814489364624,
  1.2588231563568115,
  1.554335117340088,
  1.1148098707199097,
  0.8492201566696167,
  0.9722784757614136,
  2.056586980819702,
  0.9651959538459778],
 'dev_score_history': [0.5128205128205128,
  0.48717948717948717,
  0.6410256410256411,
  0.6923076923076923,
  0.717948717948718,
  0.6410256410256411,
  0.7948717948717948,
  0.7435897435897436,
  0.6410256410256411,
  0.7948717948717948],
 'test_score': 0.7948717948717948,
 'train_loss_history': [1.56072256565094,
  1.1735487699508667,
  0.9579749226570129,
  0.6559861958026886,
  0.6486820101737976,
  0.5330302774906158,
  0.3199296295642853,
  0.2276424437761307,
  0.6377564072608948,
  0.26783226430416107]}

In [0]:
# MACRO_AVG: acc 0.8787878787878788 - f1-score 0.8233082706766918

## after fixing oversampling to be on train-only

#MACRO_AVG: acc 0.9191919191919192 - f1-score 0.8770653907496012

## trying to reproduce -- (by loading best-model.pt from annotated_sample_for_training folder. did it use Bert or Bert+Flair embedding?)
#MACRO_AVG: acc 0.7692307692307692 - f1-score 0.6489878542510121

#Aaron's score: The f1 score for GDP is 0.69

##another run-- worse -- stored in another folder?
# MACRO_AVG: acc 0.7435897435897436 - f1-score 0.6001380262249828

## another run -- Bert+Flair embedding
# MACRO_AVG: acc 0.7692307692307693 - f1-score 0.6313131313131312
#another run May26 5pm after setting random state to 42. best so far except errorneour first try
# MACRO_AVG: acc 0.7948717948717948 - f1-score 0.6691358024691358

In [24]:
## load the 2nd-stage finetuned model:

finetuned_classifier = TextClassifier.load(new_data_folder + 'best-model.pt')

2020-05-26 23:56:35,125 loading file /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotations_bnn_cbc/oversampled/best-model.pt


In [25]:
# predict same sentence from above:

sentence = Sentence("Canada's growth weakens as investment drops, consumers fade")

finetuned_classifier.predict(sentence)

print(sentence.labels) ## correct

[-1 (0.873)]


In [26]:
dir(sentence.labels)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [27]:
dir(sentence.labels[0])


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_score',
 '_value',
 'score',
 'to_dict',
 'value']

In [28]:
sentence.labels[0].value

'-1'

In [29]:
sentence.labels[0].score

0.8729730844497681

In [30]:
## error analysis

# get gold labels
#print(corpus.test[0])
print(corpus.test[0])
print(corpus.test[0].labels)



Sentence: "S & P 500 just hit another record high — and everything ’s flashing Risk On . Optimism over trade deal with China"   [− Tokens: 23  − Sentence-Labels: {'class': [1 (1.0)]}]
[1 (1.0)]


In [31]:
len(corpus.test)

26

In [0]:
gdp_test_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotations_bnn_cbc/oversampled/GDP_eva_df.csv",usecols=['title_desc_sent_1','title_desc'])

In [33]:
gdp_test_df.head()

,title_desc_sent_1,title_desc
0,1,S&P 500 just hit another record high — and eve...
1,1,Forget interest rate decisions. Loonie traders...
2,0,"With little to say on coronavirus, is Quebec's..."
3,0,Statistics are great unless they measure the w...
4,-1,"What to watch in China's GDP report: Trade, au..."


In [34]:
gdp_test_df['title_desc'].iloc[0]

'S&P 500 just hit another record high — and everything’s flashing Risk On. Optimism over trade deal with China'

In [35]:
correct = 0

for i in range(len(corpus.test)):
  print(corpus.test[i])
  

  #get predictions
  #print(gdp_test_df['title_desc'].iloc[i])
  sentence = Sentence(gdp_test_df['title_desc'].iloc[i])

  finetuned_classifier.predict(sentence)

  print(sentence.labels)

  #get gold label
  print(corpus.test[i].labels)

  #calculate correct guesses
  if sentence.labels[0].value == corpus.test[i].labels[0].value:
    correct += 1


Sentence: "S & P 500 just hit another record high — and everything ’s flashing Risk On . Optimism over trade deal with China"   [− Tokens: 23  − Sentence-Labels: {'class': [1 (1.0)]}]
[-1 (0.3915)]
[1 (1.0)]
Sentence: "Forget interest rate decisions . Loonie traders are banking on immigration to Canada . Flood of immigrants and non-permanent residents to levels not seen in a century a main driver supporting Canada 's economic expansion"   [− Tokens: 35  − Sentence-Labels: {'class': [1 (1.0)]}]
[1 (0.4258)]
[1 (1.0)]
Sentence: "With little to say on coronavirus , is Quebec 's budget ' of the future ' out of sync with the present ? . Finance minister ‚ Äôs budget bets outbreak is more public health than economic problem"   [− Tokens: 38  − Sentence-Labels: {'class': [0 (1.0)]}]
[-1 (0.926)]
[0 (1.0)]
Sentence: "Statistics are great unless they measure the wrong things : Don Pittis . Statistics Canada looks to Big Data to size up Canadians , but are the numbers biased ?"   [− Tokens: 30  

In [36]:
len(corpus.test)

26

In [37]:
correct #after train-only oversampling : 21/26 = 81% accuracy (Bert-only embedding)

# Error analysis

#reasonable wrongs

## tricky one -- econ 'stronger-than-expected". arguable golden label
# Sentence: "Consumer worries cloud best Canadian output gain in two years . Canada ’s economy recorded a stronger-than-expected rebound in the second quarter as exports recovered , but surprisingly weak consumption and business investment will cast doubts on the expansion ’s sustainability ."   [− Tokens: 42  − Sentence-Labels: {'class': [0 (1.0)]}]
# [1 (0.7199)]
# [0 (1.0)]

## doubt over golden label... not related to econ but related to interest rate
# Sentence: "Bank of Canada ’s Stephen Poloz gets chance today to put rate cut speculation on ice . Investors now see a strong chance of a cut by the Bank of Canada over the next 12 months"   [− Tokens: 36  − Sentence-Labels: {'class': [-1 (1.0)]}]
# [1 (0.8212)]
# [-1 (1.0)]

## also doubt over golden label. "improve only slightly" -- signals lower-than-expectation?
# Sentence: "Edmonton economy to improve only slightly in 2020 , city 's chief economist says . Capital city can expect ‚ Äòvery modest ‚ Äô growth , John Rose says"   [− Tokens: 29  − Sentence-Labels: {'class': [1 (1.0)]}]
# [0 (0.8225)]
# [1 (1.0)]

#unreasonable wrongs

## more related to stock market but 'optimism over trade deal' bodes well for economy. a tricky one? low confidence score
# Sentence: "S & P 500 just hit another record high — and everything ’s flashing Risk On . Optimism over trade deal with China"   [− Tokens: 23  − Sentence-Labels: {'class': [1 (1.0)]}]
# [0 (0.5386)]
# [1 (1.0)]

## recession, crisis are very negative words. how come?
# Sentence: "‘ Crisis like no other’ : IMF warns ‘ Great Lockdown’ recession will be worst in almost 100 years . In a further sign of pessimism , the IMF sketched out three alternative scenarios in which the virus lasted longer than expected , returned in 2021 or both"   [− Tokens: 48  − Sentence-Labels: {'class': [-1 (1.0)]}]
# [0 (0.9687)]
# [-1 (1.0)]



19

In [38]:
 correct #27/33 = 82% accuracy with Bert+Flair embeddings


 #before phase2 finetuning, only 66% accuracy with 1500 training examples of less GDP-specific sentiment (vs 77% acc with oversampling Kaggle dataset)

 #reasonable wrongs

## words in negative sense but meaning is neutral
# Sentence: "Why the Canadian economy seems divorced from traditional signals : Don Pittis . With housing and oil off the boil , why has n't the Canadian economy gone into free fall ?"   [− Tokens: 32  − Sentence-Labels: {'class': [0 (1.0)]}]
# [-1 (0.9096)]
# [0 (1.0)]

## not learning enough that sentiment should be Canada-specific
# Sentence: "Trump is threatening Iran with more sanctions — but what 's left to target ? . Current sanctions have already sent the country spiralling into a deep recession"   [− Tokens: 28  − Sentence-Labels: {'class': [0 (1.0)]}]
# [-1 (0.9762)]
# [0 (1.0)]

## not learning enough that sentiment should be GDP-specific
# Sentence: "If you thought August was bad , get ready for the worst month for Canadian stocks . In the past 10 years , the TSX has dropped an average 1.5 % in September"   [− Tokens: 33  − Sentence-Labels: {'class': [0 (1.0)]}]
# [-1 (0.5721)]
# [0 (1.0)]

## not that reasonable wrongs -- all predicted to the opposite extreme... problematic!

## 'stabilized' overshadowed by 'slowing' and 'weakest'? -- quite inconfident prediction
# Sentence: "China 's economy grew 6 % in fourth quarter as demand stabilized . China ’s economy stabilized last quarter after slowing to the weakest pace in almost three decades , with the first acceleration in investment since June signaling that a firmer recovery could be underway ."   [− Tokens: 47  − Sentence-Labels: {'class': [1 (1.0)]}]
# [-1 (0.5108)]
# [1 (1.0)]

## 'fastest growth' vs "trade tension"/ "threat"
# Sentence: "Oil drives Canada 's fastest economic growth spurt in a year . Canada ’s economy grew at the fastest pace in a year , further evidence of a solid expansion even as trade tensions with the U.S. remain a threat ."   [− Tokens: 41  − Sentence-Labels: {'class': [1 (1.0)]}]
# [-1 (0.6974)]
# [1 (1.0)]

## 'boom','rosy' vs 'bumpy ride' -- description balanced out but positive headline should have more weight.
# Sentence: "Mining boom to drive economic growth in territories beyond rest of Canada : report . The Conference Board of Canada says outlook rosy for Nunavut and Yukon , but N.W.T. in for a bumpy ride"   [− Tokens: 35  − Sentence-Labels: {'class': [1 (1.0)]}]
# [-1 (0.5803)]
# [1 (1.0)]

19

In [0]:

## check accuracy if only count confident predictions(condifence score > 0.7?)
confident_total = 0
confident_correct = 0

for i in range(len(corpus.test)):
  #print(corpus.test[i])
  

  #get predictions
  #print(gdp_test_df['title_desc'].iloc[i])
  sentence = Sentence(gdp_test_df['title_desc'].iloc[i])

  finetuned_classifier.predict(sentence)

  #print(sentence.labels)

  #get gold label
  #print(corpus.test[i].labels)

  #calculate correct guesses
  if sentence.labels[0].score > 0.7:
    confident_total += 1
    if sentence.labels[0].value == corpus.test[i].labels[0].value:
      confident_correct += 1

In [40]:
confident_correct

16

In [41]:
confident_total

20

In [42]:
confident_correct/confident_total ## higher accuracy at 0.8, pick only confident predictions for good visualization! also for correlation calculation?

0.8

Make Sentiment Predictions on unannotated data points

In [0]:
## get the phase2 finetuned classifier

finetuned_classifier = TextClassifier.load(new_data_folder + 'best-model.pt')


In [57]:
unannotated_gdp_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/unannotated_for_predictions/predictions_dataset_GDP_Bloomberg.csv')

unannotated_gdp_df.head() # 43 rows

,Unnamed: 0,source,title_desc,publishedAt
0,0,Bloomberg News,Economic growth stalled in February ahead of v...,2020-04-30
1,67,"{'id': 'fp-bloomberg-news', 'name': 'Bloomberg...",America’s longest economic expansion is over —...,2020-04-29
2,69,"{'id': 'fp-bloomberg-news', 'name': 'Bloomberg...","Once seen as safer than gold, Canadian real es...",2020-04-16
3,1,Bloomberg News,China won't be able to bail us out this time. ...,2020-04-16
4,71,"{'id': 'fp-bloomberg-news', 'name': 'Bloomberg...",Michael Burry of 'The Big Short' slams coronav...,2020-04-07


In [58]:
unannotated_gdp_df['title_desc'].iloc[0]

'Economic growth stalled in February ahead of virus downturn. Canada’s economy slid to a halt in February as the COVID-19 outbreak abroad dampened global growth.'

In [59]:
## add two new columns to export predictions
unannotated_gdp_df['pred_label'] = None
unannotated_gdp_df['confidence'] = None
unannotated_gdp_df.head()

,Unnamed: 0,source,title_desc,publishedAt,pred_label,confidence
0,0,Bloomberg News,Economic growth stalled in February ahead of v...,2020-04-30,None,None
1,67,"{'id': 'fp-bloomberg-news', 'name': 'Bloomberg...",America’s longest economic expansion is over —...,2020-04-29,None,None
2,69,"{'id': 'fp-bloomberg-news', 'name': 'Bloomberg...","Once seen as safer than gold, Canadian real es...",2020-04-16,None,None
3,1,Bloomberg News,China won't be able to bail us out this time. ...,2020-04-16,None,None
4,71,"{'id': 'fp-bloomberg-news', 'name': 'Bloomberg...",Michael Burry of 'The Big Short' slams coronav...,2020-04-07,None,None


In [60]:
for i in range(len(unannotated_gdp_df)):

  print(unannotated_gdp_df['title_desc'].iloc[i])
  sentence = Sentence(unannotated_gdp_df['title_desc'].iloc[i])

  finetuned_classifier.predict(sentence)

  print(sentence.labels)
  print(sentence.labels[0].value)
  print(sentence.labels[0].score)

  unannotated_gdp_df['pred_label'].iloc[i] = sentence.labels[0].value
  unannotated_gdp_df['confidence'].iloc[i] = sentence.labels[0].score




Economic growth stalled in February ahead of virus downturn. Canada’s economy slid to a halt in February as the COVID-19 outbreak abroad dampened global growth.
[-1 (0.9984)]
-1
0.9983717799186707
America’s longest economic expansion is over — and the deepest recession in at least eight decades has begun. GPD falls 4.8%, consumer spending down most since 1980, business investment plummets to 11-year low
[-1 (0.9985)]
-1
0.9984745383262634
Once seen as safer than gold, Canadian real estate braces for the 'Great Reckoning'. The real estate industry is now in a state of paralysis with households among the world's most indebted, poorly placed to weather the storm
[-1 (0.9846)]
-1
0.9846396446228027
China won't be able to bail us out this time. As bad as China's economic dive is, the slump is even more perilous for the rest of Asia. The region stands to lose its growth patron.
[-1 (0.944)]
-1
0.9439796209335327
Michael Burry of 'The Big Short' slams coronavirus lockdowns in tweetstorm. Burr

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


[-1 (0.8865)]
-1
0.8864774107933044
Canada ekes out meager growth in January before pandemic. Canada’s economy expanded less than expected in January even before the pandemic caused a virtual shutdown, reinforcing views the first quarter will be weak.
[-1 (0.999)]
-1
0.9990338087081909
Canada ekes out meagre growth in January before COVID-19 pandemic shut down economy. Gross domestic product rose 0.1 per cent from December, missing economist estimates for a 0.2 per cent gain
[-1 (0.9959)]
-1
0.9958828687667847
White House, senators strike deal on America's biggest rescue package ever — $2 trillion in spending and tax breaks. Spending, including direct payments to individuals, dwarfs the $800 billion Obama stimulus that passed five months after the 2008 financial crash
[-1 (0.9959)]
-1
0.9958914518356323
Stock losses deepen after latest U.S. stimulus deal fails. The S&P is down almost 35 per cent from its Feb. 19 record and marked its lowest close of Donald Trump’s presidency
[-1 (0.680

In [61]:
unannotated_gdp_df.head()

,Unnamed: 0,source,title_desc,publishedAt,pred_label,confidence
0,0,Bloomberg News,Economic growth stalled in February ahead of v...,2020-04-30,-1,0.998372
1,67,"{'id': 'fp-bloomberg-news', 'name': 'Bloomberg...",America’s longest economic expansion is over —...,2020-04-29,-1,0.998475
2,69,"{'id': 'fp-bloomberg-news', 'name': 'Bloomberg...","Once seen as safer than gold, Canadian real es...",2020-04-16,-1,0.98464
3,1,Bloomberg News,China won't be able to bail us out this time. ...,2020-04-16,-1,0.94398
4,71,"{'id': 'fp-bloomberg-news', 'name': 'Bloomberg...",Michael Burry of 'The Big Short' slams coronav...,2020-04-07,-1,0.886477


In [0]:
unannotated_gdp_df.to_csv("/content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/unannotated_for_predictions/unannotated_GDP_Bloomberg_predictions.csv",index=False)


In [0]:
## github master branch commit comment

#Predictions on GDP-associated Bloomberg articles

#Predictions on un-annotated GDP-associated Bloomberg articles from a model finetuned on BERT+FLAIR embeddings with devset results (MACRO_AVG: acc 0.795 and f1-score 0.669). I think the predicted labels are pretty accurate/reasonable.